In [1]:
import json
import numpy as np
from tqdm import tqdm

import sys
sys.path.append("..")


In [2]:
from helper_functions import extract_text,load_data,apply_negation,get_entity2

2024-12-19 23:19:55.844712: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1734643195.858000   46394 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1734643195.861891   46394 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-19 23:19:55.876813: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
TRAIN_PATH = '../data/raw/PIZZA_train.json'
DEV_PATH = '../data/raw/PIZZA_dev.json'
TEST_PATH = '../data/raw/PIZZA_test.json'

In [4]:
_, explanations_train, topics_train, decoupled_topics_train=map(np.array, zip(*[extract_text(data,'train') for data in load_data(TRAIN_PATH,'train')]))
_, explanations_dev, topics_dev, decoupled_topics_dev=map(np.array, zip(*[extract_text(data,'dev') for data in load_data(DEV_PATH,'dev')]))
_, explanations_test, topics_test, decoupled_topics_test=map(np.array, zip(*[extract_text(data,'test') for data in load_data(TEST_PATH,'test')]))

In [5]:
entities_dev = set([word[1:]
               for t in topics_dev for word in t.split() if word.isupper()])

entities_train = set([word[1:]
               for t in topics_train for word in t.split() if word.isupper()])

full_entities = entities_dev | entities_train

enitities_exclude_not = full_entities - {'NOT'}

negate_mapping = {}
for entity in full_entities:
    if entity != 'NOT':
        negate_mapping[entity] = "NOT_" + entity

In [6]:
negated_topics_train = np.vectorize(lambda x: apply_negation(x, negate_mapping,enitities_exclude_not))(topics_train)
negated_topics_dev = np.vectorize(lambda x: apply_negation(x, negate_mapping,enitities_exclude_not))(topics_dev)
negated_topics_test = np.vectorize(lambda x: apply_negation(x, negate_mapping,enitities_exclude_not))(topics_test)

In [8]:
final_negated_entities_pizza=np.load('../data/processed/final_negated_entities_pizza.npy',allow_pickle=True)
final_negated_entities_drink=np.load('../data/processed/final_negated_entities_drink.npy',allow_pickle=True)

In [13]:
final_negated_entities = set(final_negated_entities_pizza) | set(final_negated_entities_drink)

In [14]:
words_entities_train = [get_entity2(topic,{'DRINKORDER', 'PIZZAORDER'}, final_negated_entities) for topic in 
                           tqdm(negated_topics_train,desc="train pizza")]

words_entities_dev = [get_entity2(topic,{'DRINKORDER', 'PIZZAORDER'}, final_negated_entities) for topic in
                            tqdm(negated_topics_dev,desc="dev pizza")]

words_entities_test = [get_entity2(topic,{'DRINKORDER', 'PIZZAORDER'}, final_negated_entities) for topic in
                                tqdm(negated_topics_test,desc="test pizza")]


test pizza: 100%|██████████| 1357/1357 [00:00<00:00, 89745.67it/s]


In [15]:
np.save('../data/processed/train/words_entities_train_second.npy',np.array(words_entities_train,dtype=object))

np.save('../data/processed/dev/words_entities_dev_second.npy',np.array(words_entities_dev,dtype=object))

np.save('../data/processed/test/words_entities_test_second.npy',np.array(words_entities_test,dtype=object))


In [16]:
words_entities_dev[0]

['O',
 'O',
 'O',
 'O',
 'B-PIZZAORDER',
 'I-PIZZAORDER',
 'I-PIZZAORDER',
 'I-PIZZAORDER',
 'I-PIZZAORDER',
 'I-PIZZAORDER',
 'I-PIZZAORDER',
 'I-PIZZAORDER',
 'O',
 'B-PIZZAORDER',
 'I-PIZZAORDER',
 'I-PIZZAORDER',
 'I-PIZZAORDER',
 'I-PIZZAORDER',
 'I-PIZZAORDER',
 'I-PIZZAORDER',
 'I-PIZZAORDER',
 'O',
 'B-PIZZAORDER',
 'I-PIZZAORDER',
 'I-PIZZAORDER',
 'I-PIZZAORDER',
 'I-PIZZAORDER',
 'I-PIZZAORDER',
 'I-PIZZAORDER']